In [8]:
def func_x(x):
    return - x**5 - 2*x + 1

In [9]:
def bisection(a, b, func_x, tol=1e-5):
    if abs(a - b) < tol:
        c = (a + b) / 2
        if func_x(c) == 0:
            return c
        elif func_x(a) * func_x(c) < 0:
            return bisection(a, c, func_x)
        elif func_x(a) * func_x(c) > 0:
            return bisection(c, b, func_x)
        else:
            return "No root in (a, b)"

In [33]:
def secant_method(func, x0, x1, tol=1e-10, max_iter=100):
    x_k_minus_1 = x0
    x_k = x1
    for iteration in range(max_iter):
        f_k_minus_1 = func(x_k_minus_1)
        f_k = func(x_k)
        x_k_plus_1 = x_k - f_k * (x_k - x_k_minus_1) / (f_k - f_k_minus_1)
        if abs(x_k_plus_1 - x_k) < tol:
            return x_k_plus_1, iteration + 1
        x_k_minus_1, x_k = x_k, x_k_plus_1

In [34]:
def Newton_Raphson(func, x0, tol=1e-12, max_iter=100):
    x = x0
    dfunc = (func(x + tol) - func(x)) / tol
    for iteration in range(max_iter):
        x = x - func(x) / dfunc
        if abs(func(x)) < tol:
            return x, iteration + 1

In [35]:
%%time
bisection(-2, 2, func_x)


CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 15.7 µs


In [36]:
%%time
secant_method(func_x, 0, 0.2)

CPU times: user 37 µs, sys: 3 µs, total: 40 µs
Wall time: 44.6 µs


(0.486389035934543, 6)

In [37]:
%%time
Newton_Raphson(func_x, 0.2)

CPU times: user 33 µs, sys: 2 µs, total: 35 µs
Wall time: 38.6 µs


(0.486389035934477, 14)

In [ ]:
# oPTIM CUAR HAFM KO RANGF BUOOJC
# Toocs do hoi tu thuat toan cho cac phuong trinhf nhieu bien so
